In [242]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 
from datetime import date
import re 


In [58]:
#Connecting to Website
# http://httpbin.org/get to get user-agent fi el headers

In [445]:
def amazon_scrape():
    """ this function takes in the df we want to append data to & get the link from user then append the data into the given df"""

    url__ = input("please, paste amazon link here ")

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page = requests.get(url__, headers = headers)
    soup_1 = BeautifulSoup(page.content, "html.parser")
    # print(soup_1)

    title = soup_1.find(id='productTitle').get_text()
    # print(title)
    price = soup_1.find(id="corePrice_desktop").get_text()
    reviews = soup_1.find(id="reviewsMedley").get_text()
    
   
    date_ = date.today()    
    product_dict= {"prodcut_name":[title], "price":[price], "reviews":[reviews],"date":[date_], "url":url__}
    product_df = pd.DataFrame(product_dict)
  
    print(f"Done! check {title} data")
    return product_df
    

In [431]:
def clean_products_df(products_df_):
    """this funcation cleans product df and produce new cleaned dataframe """
    
    # convert reviews column from object type to str using astype() method '|S' means any length 
    #products_df_["reviews"] = products_df_["reviews"].astype('|S')
    products_df_["total_ratings"]= 0
    products_df_["rating_out_of_5"] = 0
    products_df_["5_stars_percent"] = 0
    products_df_["4_stars_percent"] =0 
    products_df_["3_stars_percent"] = 0
    products_df_["2_stars_percent"] = 0
    products_df_["1_stars_percent"] = 0
    
    for i in range(len(products_df_)):
        
        # clean prodcut_name column
        pr_name = products_df_["prodcut_name"][i]
        pr_name =  re.sub(r'[^a-zA-Z0-9\s]+','',pr_name) 
        pr_name = pr_name.strip()
        pr_name = re.sub(' +'," ", pr_name)
        
        products_df_.loc[i,"prodcut_name"]= pr_name
        #print(products_df_["prodcut_name"][i])
        #clean price column
        price_ = re.findall(r'[\d]*[.][\d]+', products_df_["price"][i] )
        
        products_df_.loc[i,"price"]= price_[0]
        #print(products_df_["price"][i])
        
        #extract ratings into new columns
        products_df_.loc[i, "reviews"]= str(products_df_["reviews"][i]).replace("\\n","")
        review_elements = products_df_["reviews"][i].split("How are ratings calculated?")
        total_ratings_stars = review_elements[0].split("global ratings")
        total_stars_out_of_5 = total_ratings_stars[0].split("            ")
        total_ratings_count = total_stars_out_of_5[-1]
        #print(products_df_["reviews"][i])
        
        products_df_.loc[i, "total_ratings"]= total_ratings_count.replace(",","")
        #print(products_df_["total_ratings"][i])

        rating = total_stars_out_of_5[0].replace("Customer Reviews","")
        rating = rating.split("stars")
        rating= rating[0]
        rating = re.findall(r'[\d]*[.][\d]+',rating)
        #print(rating)
        products_df_.loc[i,"rating_out_of_5"]= rating[0]
        
        stars_percent = total_ratings_stars[1].split("%")
        stars_percent = [re.sub(' +'," ",stars_percent[i]) for i in range(len(stars_percent))]
        stars_percent = [re.sub( r'[^a-zA-Z0-9\s]+',"",stars_percent[i]) for i in range(len(stars_percent))]
       # print(products_df_)

        stars_percent_list= []
        for j in range(5):
            star_j = stars_percent[j].split(" ")
            star_j_percent = star_j[-1]
            stars_percent_list.append(star_j_percent)
        
        products_df_.loc[i, "5_stars_percent"]= stars_percent_list[0]
        
        products_df_.loc[i,"4_stars_percent"]= stars_percent_list[1]
        products_df_.loc[i,"3_stars_percent"]= stars_percent_list[2]
        products_df_.loc[i,"2_stars_percent"]= stars_percent_list[3]
        products_df_.loc[i,"1_stars_percent"]= stars_percent_list[4]
    
    
    products_df_["prodcut_name"] = products_df_["prodcut_name"].astype(pd.StringDtype())
    products_df_["price"]= products_df_["price"].astype(float)
    products_df_["total_ratings"]= products_df_["total_ratings"].astype(int)
    products_df_["rating_out_of_5"]= products_df_["rating_out_of_5"].astype(float)
    products_df_["5_stars_percent"]= products_df_["5_stars_percent"].astype(float)
    products_df_["4_stars_percent"]= products_df_["4_stars_percent"].astype(float)
    products_df_["3_stars_percent"]= products_df_["3_stars_percent"].astype(float)
    products_df_["2_stars_percent"]= products_df_["2_stars_percent"].astype(float)
    products_df_["1_stars_percent"]= products_df_["1_stars_percent"].astype(float)
    
    products_= products_df_.drop(["reviews"],axis= 1)
    print("The cleaning is done you can check your dataframe ")
    return products_

# We will scrape data link by link
we will scrape 20 links (shoeses)

In [411]:
shoes_1 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/ACTIVNEW-MENS-SPORT-SHOES-WHITE/dp/B09NP25B97?ref_=Oct_d_otopr_d_21845434031&pd_rd_w=KRjIb&content-id=amzn1.sym.66034381-6399-4395-a965-d55aa520b1b2&pf_rd_p=66034381-6399-4395-a965-d55aa520b1b2&pf_rd_r=1WCZWW3PRY6N5M2P7HZQ&pd_rd_wg=9VFsd&pd_rd_r=4a26ac58-e599-44bf-9567-2aa3a3039cf9&pd_rd_i=B09NP25B97&th=1
Done! check         ACTIVNEW MEN'S SPORT SHOES        data


In [413]:
shoes_2 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Testa-Toro-Daily-Microfiber-Casual/dp/B09V1M9DDM/ref=sr_1_2?crid=1ZTMNJJXEH184&keywords=testa+toro+men%27s+daily+life+microfiber&qid=1670009597&qu=eyJxc2MiOiIwLjgwIiwicXNhIjoiMC4wMCIsInFzcCI6IjAuMDAifQ%3D%3D&sprefix=testa+toro+men%27s+daily+life+microfib%2Caps%2C115&sr=8-2
Done! check         Testa Toro Men's Daily Life Microfiber Casual Shoes        data


In [415]:
shoes_3 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/adidas-Response-Competition-Running-Sneaker/dp/B09G4JJJ9V/ref=sr_1_37?keywords=shoes+mens&qid=1669652764&qu=eyJxc2MiOiIxMS4wNiIsInFzYSI6IjEwLjI0IiwicXNwIjoiOC4wOSJ9&sprefix=shoes%2Caps%2C131&sr=8-37
Done! check         adidas Mens Run Response Super 2.0 MN Competition Running Sneaker Shoes, Color        data


In [416]:
shoes_4 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Squadra-Leather-Detail-elastic-Panel/dp/B08Y8YLKC2/ref=sr_1_30?keywords=shoes+mens&qid=1669652764&qu=eyJxc2MiOiIxMS4wNiIsInFzYSI6IjEwLjI0IiwicXNwIjoiOC4wOSJ9&sprefix=shoes%2Caps%2C131&sr=8-30
Done! check         Squadra Mesh Leather Detail Side        data


In [417]:
shoes_5 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/adidas-Runfalcon-Sneaker-Athletic-Running/dp/B0919R4HHW/ref=sr_1_7?keywords=shoes+mens&qid=1670004202&qu=eyJxc2MiOiIxMS4wNiIsInFzYSI6IjEwLjI0IiwicXNwIjoiOC4wOSJ9&sprefix=shoes%2Caps%2C602&sr=8-7
Done! check         adidas Mens Runfalcon 2.0 Sneaker Athletic MNS Running Shoe, Color Semi Night Flash/Core Black/Scarlet, Size 46.5 EU        data


In [418]:
shoes_6 = amazon_scrape()

please, paste amazon link here  https://www.amazon.eg/-/en/Squadra-Leather-Detail-elastic-Panel/dp/B08WJNMWX6/ref=sr_1_19?keywords=shoes+mens&qid=1670004202&qu=eyJxc2MiOiIxMS4wNiIsInFzYSI6IjEwLjI0IiwicXNwIjoiOC4wOSJ9&sprefix=shoes%2Caps%2C602&sr=8-19
Done! check         Squadra Mesh Leather Detail Side        data


In [419]:
shoes_7 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Adidas-DURAMO-GW4082-RUNNING-SHOES/dp/B0B4WCP4H1/ref=sr_1_46?keywords=shoes+mens&qid=1670004202&qu=eyJxc2MiOiIxMS4wNiIsInFzYSI6IjEwLjI0IiwicXNwIjoiOC4wOSJ9&sprefix=shoes%2Caps%2C602&sr=8-46
Done! check         Adidas DURAMO 10        data


In [420]:
shoes_8 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Salerno-Two-Tone-Textile-Lace-Up-Fluidstreet/dp/B08WJMLWZY/ref=sr_1_54?keywords=shoes+mens&qid=1670004202&qu=eyJxc2MiOiIxMS4wNiIsInFzYSI6IjEwLjI0IiwicXNwIjoiOC4wOSJ9&sprefix=shoes%2Caps%2C602&sr=8-54
Done! check         Salerno Mens Two-Tone Textile Lace-Up Fluidstreet, Color        data


In [421]:
shoes_9 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Converse-Fashion-Sneakers-Men-Black/dp/B002OXKQIM/ref=sr_1_51?keywords=shoes+mens&qid=1670004202&qu=eyJxc2MiOiIxMS4wNiIsInFzYSI6IjEwLjI0IiwicXNwIjoiOC4wOSJ9&sprefix=shoes%2Caps%2C602&sr=8-51
Done! check         Converse Navy Blue Fashion Sneakers For Men        data


In [422]:
shoes_10 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/ASICS-MENS-SPORTSTYLE-GEL-1090-WHITE/dp/B07SKHM34W/ref=sr_1_106?keywords=shoes+mens&qid=1670004895&qu=eyJxc2MiOiIxMS4wNiIsInFzYSI6IjEwLjI0IiwicXNwIjoiOC4wOSJ9&sprefix=shoes%2Caps%2C602&sr=8-106
Done! check         ASICS MEN'S SPORTSTYLE GEL-1090        data


In [396]:
shoes_11 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/SKECHERS-DLUX-WALKER-SPORTS-Collection/dp/B07YG1WXD4/ref=sr_1_103?keywords=men+shoes&qid=1670008321&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-103
Done! check         SKECHERS D'LUX WALKER - SPORTS Collection For MEN Color        data


In [398]:
shoes_12 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Romba-Sport-Mens-Shoes-Black/dp/B09SY7XMD1/ref=sr_1_101?keywords=men+shoes&qid=1670008321&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-101
Done! check         Romba Sport Men's Shoes        data


In [400]:
shoes_13 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Adidas-TRAINER-GX0728-RUNNING-black/dp/B096NCNHHX/ref=sr_1_123?keywords=men+shoes&qid=1670008321&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-123
Done! check         Adidas TRAINER V        data


In [401]:
shoes_14 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/adidas-RACER-GREFIV-GRESIX-GREFOU/dp/B09N3M5DSN/ref=sr_1_139?keywords=men+shoes&qid=1670008321&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-139
Done! check         adidas LITE RACER ADAPT 4.0 Men's        data


In [402]:
shoes_15 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Vans-Old-Skool-Laced-Black/dp/B01MXF6F5H/ref=sr_1_142?keywords=men+shoes&qid=1670008321&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-142
Done! check         Vans Old Skool Laced Up For Men        data


In [403]:
shoes_16 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Skechers-Mens-Walk-Motley-Color/dp/B08S7R1KDQ/ref=sr_1_148?keywords=men+shoes&qid=1670008321&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-148
Done! check         Skechers Mens Go Walk 6 Motley, Color:        data


In [404]:
shoes_17 = amazon_scrape()

please, paste amazon link here  https://www.amazon.eg/-/en/Skechers-Stand-Sneakers-Shoes-Color/dp/B07W72RM1Y/ref=sr_1_150?keywords=men%2Bshoes&qid=1670008809&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-150&th=1
Done! check         Skechers Mens Uno Stand on Air Sneakers Shoes, Color:        data


In [405]:
shoes_18 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/SPORT-Fashion-Sneakers-Men-Black/dp/B091K6NSGK/ref=sr_1_163?keywords=men+shoes&qid=1670009015&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-163
Done! check         SPORT Fashion Sneakers For Men        data


In [406]:
shoes_19 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/Salerno-Contrast-Lace-Up-Fashion-Sneakers/dp/B08WJPRWQM/ref=sr_1_177?keywords=men+shoes&qid=1670009015&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&s=fashion&sr=1-177
Done! check         Salerno Mesh Contrast Lining        data


In [407]:
shoes_20 = amazon_scrape()

please, paste amazon link here https://www.amazon.eg/-/en/UMBRO-TRAINERS-SHOES-Color-VERMILLION/dp/B09GS5V71M/ref=sr_1_15?keywords=men+shoes&qid=1670010689&qu=eyJxc2MiOiIxMS41NCIsInFzYSI6IjEwLjU1IiwicXNwIjoiOC4xNSJ9&sprefix=men+shoes%2Caps%2C131&sr=8-15
Done! check         UMBRO TRAINERS SHOES For MEN Color        data


# append all scrapped data to one dataframe

In [423]:
amazon_shoes_df = pd.concat([shoes_1, shoes_2, shoes_3,shoes_4,shoes_5,shoes_6,shoes_7,shoes_8, shoes_9, shoes_10, shoes_11, shoes_12, shoes_13,shoes_14,shoes_15,shoes_16,shoes_17,shoes_18, shoes_19])

In [424]:
amazon_shoes_df

,prodcut_name,price,reviews,date,url
0,ACTIVNEW MEN'S SPORT SHOES,\n Price: EGP299.00EGP299.00 \n\n\n\n\n\n\n\n...,Customer Reviews4.0 out of 5 stars4 out of 5\n...,2022-12-02,https://www.amazon.eg/-/en/ACTIVNEW-MENS-SPORT...
0,Testa Toro Men's Daily Life Microfiber...,\n Price: EGP185.00EGP185.00 \n\n\n\n\n\n\n\n...,Customer Reviews3.8 out of 5 stars3.8 out of 5...,2022-12-02,https://www.amazon.eg/-/en/Testa-Toro-Daily-Mi...
0,adidas Mens Run Response Super 2.0 MN ...,"\n Price: EGP1,229.00EGP1,229.00 - EGP1,679....",Customer Reviews4.1 out of 5 stars4.1 out of 5...,2022-12-02,https://www.amazon.eg/-/en/adidas-Response-Com...
0,Squadra Mesh Leather Detail Side,\n Price: EGP69.00EGP69.00 - EGP79.00EGP79.0...,Customer Reviews2.9 out of 5 stars2.9 out of 5...,2022-12-02,https://www.amazon.eg/-/en/Squadra-Leather-Det...
0,adidas Mens Runfalcon 2.0 Sneaker Athl...,"\n Price: EGP1,629.00EGP1,629.00 \n\n\n\n\n\n...",Customer Reviews4.4 out of 5 stars4.4 out of 5...,2022-12-02,https://www.amazon.eg/-/en/adidas-Runfalcon-Sn...
0,Squadra Mesh Leather Detail Side,\n Price: EGP79.00EGP79.00 \n\n\n\n\n\n\n\n \...,Customer Reviews2.9 out of 5 stars2.9 out of 5...,2022-12-02,https://www.amazon.eg/-/en/Squadra-Leather-De...
0,Adidas DURAMO 10,"\n Price: EGP1,879.00EGP1,879.00 \n\n\n\n\n\n...",Customer Reviews4.0 out of 5 stars4 out of 5\n...,2022-12-02,https://www.amazon.eg/-/en/Adidas-DURAMO-GW408...
0,Salerno Mens Two-Tone Textile Lace-Up ...,\n Price: EGP119.00EGP119.00 - EGP139.00EGP1...,Customer Reviews4.0 out of 5 stars4 out of 5\n...,2022-12-02,https://www.amazon.eg/-/en/Salerno-Two-Tone-Te...
0,Converse Navy Blue Fashion Sneakers Fo...,"\n Price: EGP1,449.00EGP1,449.00 - EGP1,699....",Customer Reviews4.5 out of 5 stars4.5 out of 5...,2022-12-02,https://www.amazon.eg/-/en/Converse-Fashion-Sn...
0,ASICS MEN'S SPORTSTYLE GEL-1090,"\n Price: EGP1,659.00EGP1,659.00 - EGP2,249....",Customer Reviews4.3 out of 5 stars4.3 out of 5...,2022-12-02,https://www.amazon.eg/-/en/ASICS-MENS-SPORTSTY...


# Create new csv with the collected data 

In [440]:
#amazon_shoes_df.to_csv(f"E:\\study\\DA_Projects\\Python_projects\\Amazon_project\\data\\amazon_shoes_df_{date.today()}.csv", index= False)

# Cleaning 

In [433]:
products_1 = pd.read_csv(r"E:\study\DA_Projects\Python_projects\Amazon_project\data\amazon_shoes_df_2022-12-02.csv")

In [434]:
products_1


,prodcut_name,price,reviews,date,url
0,ACTIVNEW MEN'S SPORT SHOES,\n Price: EGP299.00EGP299.00 \n\n\n\n\n\n\n\n...,Customer Reviews4.0 out of 5 stars4 out of 5\n...,2022-12-02,https://www.amazon.eg/-/en/ACTIVNEW-MENS-SPORT...
1,Testa Toro Men's Daily Life Microfiber...,\n Price: EGP185.00EGP185.00 \n\n\n\n\n\n\n\n...,Customer Reviews3.8 out of 5 stars3.8 out of 5...,2022-12-02,https://www.amazon.eg/-/en/Testa-Toro-Daily-Mi...
2,adidas Mens Run Response Super 2.0 MN ...,"\n Price: EGP1,229.00EGP1,229.00 - EGP1,679....",Customer Reviews4.1 out of 5 stars4.1 out of 5...,2022-12-02,https://www.amazon.eg/-/en/adidas-Response-Com...
3,Squadra Mesh Leather Detail Side,\n Price: EGP69.00EGP69.00 - EGP79.00EGP79.0...,Customer Reviews2.9 out of 5 stars2.9 out of 5...,2022-12-02,https://www.amazon.eg/-/en/Squadra-Leather-Det...
4,adidas Mens Runfalcon 2.0 Sneaker Athl...,"\n Price: EGP1,629.00EGP1,629.00 \n\n\n\n\n\n...",Customer Reviews4.4 out of 5 stars4.4 out of 5...,2022-12-02,https://www.amazon.eg/-/en/adidas-Runfalcon-Sn...
5,Squadra Mesh Leather Detail Side,\n Price: EGP79.00EGP79.00 \n\n\n\n\n\n\n\n \...,Customer Reviews2.9 out of 5 stars2.9 out of 5...,2022-12-02,https://www.amazon.eg/-/en/Squadra-Leather-De...
6,Adidas DURAMO 10,"\n Price: EGP1,879.00EGP1,879.00 \n\n\n\n\n\n...",Customer Reviews4.0 out of 5 stars4 out of 5\n...,2022-12-02,https://www.amazon.eg/-/en/Adidas-DURAMO-GW408...
7,Salerno Mens Two-Tone Textile Lace-Up ...,\n Price: EGP119.00EGP119.00 - EGP139.00EGP1...,Customer Reviews4.0 out of 5 stars4 out of 5\n...,2022-12-02,https://www.amazon.eg/-/en/Salerno-Two-Tone-Te...
8,Converse Navy Blue Fashion Sneakers Fo...,"\n Price: EGP1,449.00EGP1,449.00 - EGP1,699....",Customer Reviews4.5 out of 5 stars4.5 out of 5...,2022-12-02,https://www.amazon.eg/-/en/Converse-Fashion-Sn...
9,ASICS MEN'S SPORTSTYLE GEL-1090,"\n Price: EGP1,659.00EGP1,659.00 - EGP2,249....",Customer Reviews4.3 out of 5 stars4.3 out of 5...,2022-12-02,https://www.amazon.eg/-/en/ASICS-MENS-SPORTSTY...


In [435]:
clean_pr_df_no_reviews = clean_products_df(products_1)

The cleaning is done you can check your dataframe 


In [436]:
clean_pr_df_no_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   prodcut_name     19 non-null     string 
 1   price            19 non-null     float64
 2   date             19 non-null     object 
 3   url              19 non-null     object 
 4   total_ratings    19 non-null     int32  
 5   rating_out_of_5  19 non-null     float64
 6   5_stars_percent  19 non-null     float64
 7   4_stars_percent  19 non-null     float64
 8   3_stars_percent  19 non-null     float64
 9   2_stars_percent  19 non-null     float64
 10  1_stars_percent  19 non-null     float64
dtypes: float64(7), int32(1), object(2), string(1)
memory usage: 1.7+ KB


In [437]:
clean_pr_df_no_reviews

,prodcut_name,price,date,url,total_ratings,rating_out_of_5,5_stars_percent,4_stars_percent,3_stars_percent,2_stars_percent,1_stars_percent
0,ACTIVNEW MENS SPORT SHOES,299.0,2022-12-02,https://www.amazon.eg/-/en/ACTIVNEW-MENS-SPORT...,85,4.0,50.0,22.0,16.0,4.0,8.0
1,Testa Toro Mens Daily Life Microfiber Casual S...,185.0,2022-12-02,https://www.amazon.eg/-/en/Testa-Toro-Daily-Mi...,235,3.8,62.0,4.0,7.0,8.0,18.0
2,adidas Mens Run Response Super 20 MN Competiti...,229.0,2022-12-02,https://www.amazon.eg/-/en/adidas-Response-Com...,92,4.1,62.0,15.0,7.0,4.0,13.0
3,Squadra Mesh Leather Detail Side,69.0,2022-12-02,https://www.amazon.eg/-/en/Squadra-Leather-Det...,156,2.9,26.0,11.0,19.0,10.0,33.0
4,adidas Mens Runfalcon 20 Sneaker Athletic MNS ...,629.0,2022-12-02,https://www.amazon.eg/-/en/adidas-Runfalcon-Sn...,5767,4.4,66.0,21.0,8.0,2.0,4.0
5,Squadra Mesh Leather Detail Side,79.0,2022-12-02,https://www.amazon.eg/-/en/Squadra-Leather-De...,156,2.9,26.0,11.0,19.0,10.0,33.0
6,Adidas DURAMO 10,879.0,2022-12-02,https://www.amazon.eg/-/en/Adidas-DURAMO-GW408...,48,4.0,55.0,19.0,11.0,0.0,0.0
7,Salerno Mens TwoTone Textile LaceUp Fluidstree...,119.0,2022-12-02,https://www.amazon.eg/-/en/Salerno-Two-Tone-Te...,70,4.0,42.0,33.0,12.0,5.0,8.0
8,Converse Navy Blue Fashion Sneakers For Men,449.0,2022-12-02,https://www.amazon.eg/-/en/Converse-Fashion-Sn...,8609,4.5,73.0,14.0,5.0,2.0,6.0
9,ASICS MENS SPORTSTYLE GEL1090,659.0,2022-12-02,https://www.amazon.eg/-/en/ASICS-MENS-SPORTSTY...,79,4.3,66.0,18.0,6.0,1.0,7.0


# save the new cleaned dataframe into a new csv with todays date 

In [444]:
#clean_pr_df_no_reviews.to_csv(f"E:\\study\\DA_Projects\\Python_projects\\Amazon_project\\data\\cleaned_amazon_shoes_df_no_reviews_{date.today()}.csv",index= False)

In [3]:
text = "PYTHONSQLRSTATISTICAL ANALYSISHYPOTHESIS TESTING ( STATISTICAL TESTING )BIGQUERYTABLEAUPOWERPIMACHINE LEARNINGSCIKIT-LEARNNLPWEB CRAWLINGTIDYVERSEGGPLOT2SHEETS - EXCELSOCIAL MEDIA ANALYTICSDATA VISUALIZATIONDEVELOPING EFFECTIVE PRESENTATIONSDEVELOPING REPORTSPROBLEM SOLVINGPRESENTATION AND COMMUNICATION SKILLS"

In [6]:
text.lower()

'pythonsqlrstatistical analysishypothesis testing ( statistical testing )bigquerytableaupowerpimachine learningscikit-learnnlpweb crawlingtidyverseggplot2sheets - excelsocial media analyticsdata visualizationdeveloping effective presentationsdeveloping reportsproblem solvingpresentation and communication skills'